In [1]:
import cv2
import numpy as np
import json
from tqdm import tqdm

In [2]:
video_name="ID-5"

In [3]:
annotations = json.load(open(f'./{video_name}_video_ann.json'))

# Generazione video con annotazioni

In [4]:
cap = cv2.VideoCapture(f'./test/{video_name}.avi')


frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(f'{video_name}_with_annotations.avi', fourcc, fps, (frame_width, frame_height))

square_size = 20  # Width and height of the square
color = (0, 255, 0)  # Green color for the square

for frame_index in tqdm(range(0, frame_count)):
    ret, frame = cap.read()
    if not ret:
        break
        
    coord = annotations[f'{frame_index:04d}.png']
    x, y = coord['x'], coord['y']
    if x>=0 and y>=0:
        top_left = (round(x - square_size // 2), round(y - square_size // 2))
        bottom_right = (round(x + square_size // 2), round(y + square_size // 2))
        cv2.rectangle(frame, top_left, bottom_right, color, 2)
        conf = round(coord['conf'], 2)
        cv2.putText(frame, f'Ball - {conf}', (round(x)-10, round(y)-15), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
    out.write(frame)
    
cap.release()
out.release()


00%|█████████████████████████████████████████████████████████████████████████████▉| 2999/3002 [00:57<00:00, 51.88it/s]

# Generazione video con traccia

In [5]:
current_color = (0, 255, 0)  # Green for current point
line_color = (0, 0, 255)     # Blue for the polyline

cap = cv2.VideoCapture(f'./test/{video_name}.avi')

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

out = cv2.VideoWriter(f'{video_name}_with_tracks.avi', fourcc, fps, (frame_width, frame_height))

square_size = 20  # Width and height of the square
color = (0, 255, 0)  # Green color for the square

for frame_index in tqdm(range(0, frame_count)):
    ret, frame = cap.read()
    if not ret:
        break
        
    coord = annotations[f'{frame_index:04d}.png']
    x, y = coord['x'], coord['y']
    if x>=0 and y>=0:
        top_left = (round(x - square_size // 2), round(y - square_size // 2))
        bottom_right = (round(x + square_size // 2), round(y + square_size // 2))
        cv2.rectangle(frame, top_left, bottom_right, color, 2)
        points = []
        for next_frame in range(frame_index, min(frame_index + 11, frame_count)):
            frame_key = f'{next_frame:04d}.png'
            x, y = annotations[frame_key]['x'], annotations[frame_key]['y']
            if x<0 and y<0:
                break
            points.append((round(x), round(y)))
        if len(points)>0:
            points_np = np.array(points, dtype=np.int32)  # Convert list to numpy array of type int32
            points_np = points_np.reshape((-1, 1, 2))  # Reshape to the format expected by polylines
            cv2.polylines(frame, [points_np], isClosed=False, color=line_color, thickness=5)
    out.write(frame)
    
cap.release()
out.release()


00%|█████████████████████████████████████████████████████████████████████████████▉| 2999/3002 [00:58<00:00, 51.30it/s]